In [15]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import funclib


%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')

train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [17]:
# 划分特征与标签
train_ds = train.iloc[:,13:1913]
test_ds = test.iloc[:,13:1913]
train_lb = train.ec_label
test_lb = test.ec_label

#最大的EC数目
max_ec = len(set(list(train.ec_number) + list(test.ec_number)))

#unirep的数据精度
unit_thres = 4
train_ds = train_ds.round(unit_thres)
test_ds = test_ds.round(unit_thres)

In [22]:
train_lb.to_csv('../../baselines/slice/Sandbox/Data/sprot/train_lb1.txt', index=0, header=0)

In [18]:
#写入训练标签文件

file_write_obj = open("../../baselines/slice/Sandbox/Data/sprot/train_lb.txt", 'w')
file_write_obj.writelines('{0} {1}\n'.format(len(train), max_ec))
for var in train_lb:
    file_write_obj.writelines('{0}:1\n'.format(var))
file_write_obj.close()

# 写入测试标签文件
file_write_obj = open("../../baselines/slice/Sandbox/Data/sprot/test_lb.txt", 'w')
file_write_obj.writelines('{0} {1}\n'.format(len(test), max_ec))
for var in test_lb:
    file_write_obj.writelines('{0}:1\n'.format(var))
file_write_obj.close()


In [19]:
# 将数据字符串化
test_ds_str = test_ds.astype(str)
train_ds_str = train_ds.astype(str)


In [20]:
# 用空格连城字符串
train_str = train_ds_str.apply(lambda x: ' '.join(x), axis=1)
test_str = test_ds_str.apply(lambda x: ' '.join(x), axis=1)

In [21]:
# 写入训练文件
file_write_obj = open("../../baselines/slice/Sandbox/Data/sprot/train.txt", 'w')
file_write_obj.writelines('{0} {1}\n'.format(len(train), 1900))
counter =0
for var in train_str:
    file_write_obj.writelines('{0}\n'.format(var))
file_write_obj.close()

#写测试文件
file_write_obj = open("../../baselines/slice/Sandbox/Data/sprot/test.txt", 'w')
file_write_obj.writelines('{0} {1}\n'.format(len(test), 1900))
for var in test_str:
    file_write_obj.writelines('{0}\n'.format(var))
file_write_obj.close()